In [ ]:
import tensorflow as tf
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import sys
import numpy as np

from datetime import datetime as dt
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score


W = H = 338
N_CLASSES = 11
autotune = tf.data.experimental.AUTOTUNE

features = {
    'StudyInstanceUID': tf.io.FixedLenFeature([], tf.string),
    'image': tf.io.FixedLenFeature([], tf.string),
}


target_cols = [
    'CVC - Abnormal',
    'CVC - Borderline',
    'CVC - Normal',
    'ETT - Abnormal',
    'ETT - Borderline',
    'ETT - Normal',
    'NGT - Abnormal',
    'NGT - Borderline',
    'NGT - Incompletely Imaged',
    'NGT - Normal',
    'Swan Ganz Catheter Present'
]

label_list = list(features.keys())[:11]

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

test_tfrecords_dir = "../input/ranzcr-clip-catheter-line-classification/test_tfrecords"
weight_dir = "../input/cassava2020weights"
test_tfrecords = os.listdir(test_tfrecords_dir)
model_map = {
    "efficientb3": [tf.keras.applications.EfficientNetB3, weight_dir + "/ranzcr_efficientb3.h5"],
    "efficientb5": [tf.keras.applications.EfficientNetB5, weight_dir + "/ranzcr_efficientb5.h5"],
    "efficientb7": [tf.keras.applications.EfficientNetB7, weight_dir + "/ranzcr_efficientb7.h5"],
    "resnet101": [tf.keras.applications.ResNet101,  weight_dir + "/ranzcr_resnet101.h5"],
}

In [ ]:
def parse_example(sample):
    sample= tf.io.parse_single_example(sample, features)
    image = tf.image.decode_png(sample["image"], channels=3)
    image = tf.image.resize(image, (H, W))

    image_id = sample["StudyInstanceUID"]
    
    return image, image_id

def preprocess(images, labels):
    images = tf.cast(images, tf.float32)
    return images, labels
    return (images - mean) / std, labels

def get_count(fname):
    return 1868 if "15-1881.tfrec" in fname else 1881

def get_tfrecord(indices):
    files = [test_tfrecords_dir + "/"+ train_tfrecords[i] for i in indices]
    sizes = [get_count(file) for file in files]\

    return files, sum(sizes)

def get_model(
    base_model,
    baseline_weight=None,
    init_weight=None,
    lr=0.001,
    optimizer=tf.optimizers.Adam):
    base_model = base_model(include_top=False,
                           input_shape=(W, H, 3),
                           pooling="avg",
                           weights=baseline_weight)
    base_out = base_model.output
    out = tf.keras.layers.Dropout(0.3)(base_out)
    out = tf.keras.layers.Dense(N_CLASSES, activation="sigmoid")(out)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=out)

    model.compile(optimizer=optimizer(learning_rate=lr),
                loss="binary_crossentropy",
                metrics=[tf.keras.metrics.AUC()])
    if init_weight:
        try:
            model.load_weights(init_weight)
            print(f"Weight loaded from {init_weight}")
        except Exception as e:
            print(f"Load weight from {init_weight} failed, {e}")
    return model

In [ ]:
files = [test_tfrecords_dir + "/"+ c for c in test_tfrecords]
test_data = tf.data.TFRecordDataset(files)
test_data = test_data.map(parse_example, num_parallel_calls=autotune)
test_data = test_data.map(preprocess)
test_data = test_data.prefetch(1)

###  Display some images

In [ ]:
def show_samples(dataset):
    rows = cols = 2
    fig = plt.figure(figsize=(15,15))
    for i, (img, label) in enumerate(dataset.shuffle(100).take(rows * cols)):
        fig.add_subplot(rows, cols, i + 1)
        plt.imshow(img/255)
    plt.show()
    
show_samples(test_data)

In [ ]:
base_mode, weight_path = model_map["resnet101"]
model = get_model(base_mode, init_weight=weight_path)

In [ ]:
preds = []
image_ids = []
test_df = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')[0:0]

def TTA(image):
    return tf.stack([
        image,
        tf.image.flip_left_right(image),
    ])

c = 0
use_tta = False

for image, img_id in test_data:
    img_id = str(img_id.numpy()).replace("b", "").replace("'", "")
    image_ids.append(img_id)
    
    if use_tta:
    
        tta_imgs = TTA(image)

        pred = model.predict_on_batch(tta_imgs).tolist()
        pred = np.max(pred, axis=0)
    else:
        pred = model.predict(image[tf.newaxis, ...]).tolist()[0]
        
    preds.append(pred)
    
    c += 1
    if c % 1000 == 0:
        print(c, "/3582")
    
preds
test_df[target_cols] = preds
test_df[["StudyInstanceUID"]] = image_ids
test_df.to_csv('submission.csv', index=False)
test_df.head()